# Data Analysis with dimtensor

This notebook demonstrates real-world data analysis workflows using dimtensor:

1. **CODATA Physical Constants** - Load precise physical constants with uncertainties
2. **Exoplanet Data Analysis** - Analyze NASA exoplanet archive data
3. **Unit Conversions** - Convert between different unit systems
4. **Statistical Analysis** - Compute statistics while preserving units
5. **Visualization** - Create unit-aware plots
6. **Data Export** - Save and reload data with units

We'll use real NASA exoplanet data when available, with a synthetic fallback for offline use.

In [ ]:
# Required imports
import numpy as np
import dimtensor as dt
from dimtensor import DimArray
from dimtensor.units import m, kg, s, AU, day, year
import dimtensor.constants as const

# Optional imports for data analysis and visualization
try:
    import pandas as pd
    HAS_PANDAS = True
except ImportError:
    HAS_PANDAS = False
    print("Warning: pandas not available")

try:
    import matplotlib.pyplot as plt
    HAS_MATPLOTLIB = True
except ImportError:
    HAS_MATPLOTLIB = False
    print("Warning: matplotlib not available")

print(f"dimtensor version: {dt.__version__}")
print(f"NumPy version: {np.__version__}")

## Section 1: CODATA Physical Constants

dimtensor provides access to CODATA 2022 physical constants with full metadata including uncertainties and references.

In [ ]:
# Load fundamental universal constants
c = const.c  # Speed of light
G = const.G  # Gravitational constant
h = const.h  # Planck constant
hbar = const.hbar  # Reduced Planck constant

print("Universal Constants:")
print(f"Speed of light: {c}")
print(f"Gravitational constant: {G}")
print(f"Planck constant: {h}")
print(f"Reduced Planck constant: {hbar}")

In [ ]:
# Load particle masses
m_e = const.m_e  # Electron mass
m_p = const.m_p  # Proton mass
m_n = const.m_n  # Neutron mass

print("Particle Masses:")
print(f"Electron mass: {m_e}")
print(f"Proton mass: {m_p}")
print(f"Neutron mass: {m_n}")
print(f"\nProton/electron mass ratio: {(m_p / m_e).to_base()}")

In [ ]:
# Demonstrate dimensional correctness with constants
# Calculate Compton wavelength: λ_C = h / (m_e * c)
lambda_C = h / (m_e * c)
print(f"Electron Compton wavelength: {lambda_C}")
print(f"Dimension: {lambda_C.unit.dimension}")

# Convert to nanometers for readability
from dimtensor.units import nm
print(f"In nanometers: {lambda_C.to(nm)}")

In [ ]:
# Show how dimensional analysis catches errors
try:
    # This should fail: can't add mass and length
    invalid = m_e + lambda_C
except Exception as e:
    print(f"Error caught (as expected): {type(e).__name__}")
    print(f"Message: {e}")

## Section 2: Loading Exoplanet Data

We'll load exoplanet data from NASA's Exoplanet Archive. If the download fails, we'll use synthetic data as a fallback.

In [ ]:
# Create synthetic exoplanet data as fallback
def create_synthetic_exoplanet_data():
    """Generate realistic synthetic exoplanet data for offline use."""
    np.random.seed(42)
    n_planets = 25
    
    # Planet names
    names = [f"Kepler-{100+i}b" for i in range(n_planets)]
    
    # Masses: log-normal distribution (0.1 to 10 Jupiter masses)
    log_masses = np.random.normal(0, 1.2, n_planets)
    masses_mjup = np.exp(log_masses) * 0.5
    
    # Orbital distances: log-normal (0.01 to 5 AU)
    log_distances = np.random.normal(-0.5, 1.5, n_planets)
    distances_au = np.exp(log_distances) * 0.3
    
    # Orbital periods from Kepler's 3rd law: P² ∝ a³
    # P (years) ≈ a (AU)^1.5 for solar-mass stars
    periods_days = (distances_au ** 1.5) * 365.25
    
    # Radii: correlation with mass (R ∝ M^0.5 approximately)
    radii_rjup = (masses_mjup ** 0.5) * np.random.normal(1, 0.1, n_planets)
    
    return {
        'name': names,
        'mass_mjup': masses_mjup,
        'distance_au': distances_au,
        'period_days': periods_days,
        'radius_rjup': radii_rjup
    }

# Try to load real data, fall back to synthetic
try:
    if HAS_PANDAS:
        # In a real scenario, you would download from NASA Exoplanet Archive
        # url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+pl_name,pl_bmassj,pl_orbsmax,pl_orbper,pl_radj+from+ps+where+pl_bmassj+is+not+null&format=csv"
        # df = pd.read_csv(url)
        raise ImportError("Using synthetic data for this demo")
    else:
        raise ImportError("pandas not available")
except Exception:
    print("Using synthetic exoplanet data (offline mode)")
    data = create_synthetic_exoplanet_data()
    if HAS_PANDAS:
        df = pd.DataFrame(data)
    else:
        df = data

if HAS_PANDAS:
    print(f"\nLoaded {len(df)} exoplanets")
    print(df.head())
else:
    print(f"\nLoaded {len(data['name'])} exoplanets")
    print(f"First planet: {data['name'][0]}")

In [ ]:
# Convert pandas columns to DimArray with proper units
from dimtensor.units import M_jup, R_jup

if HAS_PANDAS:
    # Extract data with units
    masses = DimArray(df['mass_mjup'].values, M_jup)
    distances = DimArray(df['distance_au'].values, AU)
    periods = DimArray(df['period_days'].values, day)
    radii = DimArray(df['radius_rjup'].values, R_jup)
    names = df['name'].values
else:
    # Use dict data
    masses = DimArray(data['mass_mjup'], M_jup)
    distances = DimArray(data['distance_au'], AU)
    periods = DimArray(data['period_days'], day)
    radii = DimArray(data['radius_rjup'], R_jup)
    names = data['name']

print("Data converted to DimArrays:")
print(f"Masses: {masses[:5]}")
print(f"Distances: {distances[:5]}")
print(f"Periods: {periods[:5]}")

In [ ]:
# Show unit metadata is preserved
print(f"Mass unit: {masses.unit}")
print(f"Mass dimension: {masses.unit.dimension}")
print(f"Distance unit: {distances.unit}")
print(f"Distance dimension: {distances.unit.dimension}")
print(f"Period unit: {periods.unit}")
print(f"Period dimension: {periods.unit.dimension}")

In [ ]:
# Display summary statistics
print("Dataset Summary:")
print(f"Number of exoplanets: {len(masses)}")
print(f"Mass range: {masses.min()} to {masses.max()}")
print(f"Distance range: {distances.min()} to {distances.max()}")
print(f"Period range: {periods.min()} to {periods.max()}")

## Section 3: Unit Conversions

dimtensor makes unit conversions easy and safe. Let's convert our exoplanet data between different unit systems.

In [ ]:
# Convert planetary masses between different units
from dimtensor.units import M_earth

# Jupiter masses → Earth masses → kilograms
masses_mearth = masses.to(M_earth)
masses_kg = masses.to(kg)

print("Mass Conversions (first 5 planets):")
for i in range(5):
    print(f"{names[i]:15s}: {masses[i]:8.2f} = {masses_mearth[i]:8.1f} = {masses_kg[i]:.2e}")

In [ ]:
# Convert orbital distances
from dimtensor.units import km

distances_m = distances.to(m)
distances_km = distances.to(km)

print("\nDistance Conversions (first 5 planets):")
for i in range(5):
    print(f"{names[i]:15s}: {distances[i]:7.3f} = {distances_km[i]:.2e}")

In [ ]:
# Convert orbital periods
periods_years = periods.to(year)
periods_seconds = periods.to(s)

print("\nPeriod Conversions (first 5 planets):")
for i in range(5):
    print(f"{names[i]:15s}: {periods[i]:8.1f} = {periods_years[i]:7.3f} = {periods_seconds[i]:.2e}")

In [ ]:
# Demonstrate automatic unit simplification
# Calculate momentum-like quantity: mass × distance / time
from dimtensor.units import N

# Force units: kg⋅m/s² should simplify to N (Newtons)
force_example = DimArray([1.0], kg) * DimArray([1.0], m) / DimArray([1.0], s**2)
print(f"\nAutomatic simplification:")
print(f"1 kg⋅m/s² = {force_example[0]}")
print(f"Is this 1 Newton? {force_example.to(N)[0]}")

In [ ]:
# Show error when trying incompatible conversions
try:
    # Can't convert mass to distance!
    invalid = masses.to(m)
except Exception as e:
    print(f"\nError caught (as expected): {type(e).__name__}")
    print(f"Message: {e}")

## Section 4: Statistical Analysis

Compute statistics on exoplanet data while preserving units throughout.

In [ ]:
# Basic statistics on orbital periods
mean_period = periods.mean()
std_period = periods.std()
median_period = DimArray([np.median(periods._data)], periods.unit)

print("Orbital Period Statistics:")
print(f"Mean: {mean_period}")
print(f"Std Dev: {std_period}")
print(f"Median: {median_period[0]}")
print(f"\nIn years:")
print(f"Mean: {mean_period.to(year)}")
print(f"Median: {median_period.to(year)[0]}")

In [ ]:
# Find extreme values
min_mass_idx = np.argmin(masses._data)
max_mass_idx = np.argmax(masses._data)

print("\nMass Extremes:")
print(f"Lightest planet: {names[min_mass_idx]}")
print(f"  Mass: {masses[min_mass_idx]} = {masses_mearth[min_mass_idx]:.1f}")
print(f"\nHeaviest planet: {names[max_mass_idx]}")
print(f"  Mass: {masses[max_mass_idx]} = {masses_mearth[max_mass_idx]:.1f}")

In [ ]:
# Calculate derived quantities: surface gravity g = G*M/R²
# Surface gravity for each planet
surface_gravity = (G * masses) / (radii ** 2)

# Convert to m/s² for comparison with Earth (g_earth ≈ 9.8 m/s²)
from dimtensor.units import m_per_s2
gravity_ms2 = surface_gravity.to(m_per_s2)

print("\nSurface Gravity (first 5 planets):")
for i in range(5):
    g_ratio = gravity_ms2._data[i] / 9.8
    print(f"{names[i]:15s}: {gravity_ms2[i]:7.2f} ({g_ratio:.2f}× Earth)")

In [ ]:
# Calculate escape velocity: v_esc = sqrt(2*G*M/R)
escape_velocity = ((2 * G * masses) / radii) ** 0.5

# Convert to km/s
from dimtensor.units import km_per_s
v_esc_kms = escape_velocity.to(km_per_s)

print("\nEscape Velocity (first 5 planets):")
for i in range(5):
    print(f"{names[i]:15s}: {v_esc_kms[i]:.1f}")

# Earth's escape velocity is ~11.2 km/s for reference
print("\n(Earth's escape velocity is ~11.2 km/s)")

In [ ]:
# Filter data by physical criteria
# Find "hot Jupiters" - massive planets close to their stars
# Criteria: mass > 0.3 M_jup AND distance < 0.1 AU
hot_jupiter_mask = (masses._data > 0.3) & (distances._data < 0.1)
n_hot_jupiters = np.sum(hot_jupiter_mask)

print(f"\nHot Jupiters (M > 0.3 M_jup, a < 0.1 AU): {n_hot_jupiters}")
if n_hot_jupiters > 0:
    print("Names:")
    for name in names[hot_jupiter_mask]:
        print(f"  - {name}")

In [ ]:
# Verify Kepler's 3rd law: P² ∝ a³ (for solar-mass stars)
# Calculate a³/P² for each planet (should be roughly constant)
kepler_ratio = (distances ** 3) / (periods ** 2)

# This has dimensions of L³/T², convert to AU³/year²
from dimtensor.units import Unit, Dimension
au3_per_yr2 = Unit(Dimension(L=3, T=-2), AU._scale**3 / year._scale**2, "AU³/yr²")
kepler_au3yr2 = kepler_ratio.to(au3_per_yr2)

print("\nKepler's 3rd Law Check (a³/P²):")
print(f"Mean: {kepler_au3yr2.mean():.3f}")
print(f"Std:  {kepler_au3yr2.std():.3f}")
print("\n(For solar-mass stars, this should be ~1.0 AU³/yr²)")

## Section 5: Visualization

Create publication-quality plots with unit-aware axis labels.

In [ ]:
if HAS_MATPLOTLIB:
    # Mass-radius relationship
    plt.figure(figsize=(10, 6))
    plt.scatter(masses_mearth._data, radii._data, alpha=0.6, s=100)
    plt.xlabel(f'Mass ({M_earth})', fontsize=12)
    plt.ylabel(f'Radius ({R_jup})', fontsize=12)
    plt.title('Exoplanet Mass-Radius Relationship', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("matplotlib not available - skipping plots")

In [ ]:
if HAS_MATPLOTLIB:
    # Orbital period distribution
    plt.figure(figsize=(10, 6))
    plt.hist(periods_years._data, bins=15, alpha=0.7, edgecolor='black')
    plt.xlabel(f'Orbital Period ({year})', fontsize=12)
    plt.ylabel('Number of Planets', fontsize=12)
    plt.title('Distribution of Orbital Periods', fontsize=14)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

In [ ]:
if HAS_MATPLOTLIB:
    # Semi-major axis vs period (Kepler's 3rd law)
    plt.figure(figsize=(10, 6))
    plt.loglog(distances._data, periods_years._data, 'o', alpha=0.6, s=100)
    
    # Overlay theoretical P = a^1.5 line
    a_theory = np.logspace(-1.5, 0.7, 50)
    p_theory = a_theory ** 1.5
    plt.loglog(a_theory, p_theory, 'r--', label='P ∝ a^1.5', linewidth=2)
    
    plt.xlabel(f'Semi-major Axis ({AU})', fontsize=12)
    plt.ylabel(f'Orbital Period ({year})', fontsize=12)
    plt.title("Kepler's Third Law", fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
if HAS_MATPLOTLIB:
    # Surface gravity comparison
    plt.figure(figsize=(12, 6))
    sorted_idx = np.argsort(gravity_ms2._data)[::-1][:10]  # Top 10 by gravity
    
    x_pos = np.arange(len(sorted_idx))
    plt.bar(x_pos, gravity_ms2._data[sorted_idx])
    plt.axhline(y=9.8, color='r', linestyle='--', label='Earth gravity', linewidth=2)
    plt.xticks(x_pos, [names[i][:12] for i in sorted_idx], rotation=45, ha='right')
    plt.ylabel(f'Surface Gravity ({m_per_s2})', fontsize=12)
    plt.title('Top 10 Planets by Surface Gravity', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

In [ ]:
if HAS_MATPLOTLIB:
    # Escape velocity vs mass
    plt.figure(figsize=(10, 6))
    plt.scatter(masses_mearth._data, v_esc_kms._data, alpha=0.6, s=100, c=distances._data, 
                cmap='viridis')
    cbar = plt.colorbar()
    cbar.set_label(f'Orbital Distance ({AU})', fontsize=11)
    plt.xlabel(f'Mass ({M_earth})', fontsize=12)
    plt.ylabel(f'Escape Velocity ({km_per_s})', fontsize=12)
    plt.title('Escape Velocity vs Mass', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Section 6: Data Export and Serialization

dimtensor supports saving and loading data with units preserved through JSON, pandas, and HDF5.

In [ ]:
# Save to JSON
import json
import tempfile
import os

# Create temporary directory for exports
temp_dir = tempfile.mkdtemp()
print(f"Export directory: {temp_dir}")

# Save first 5 planets to JSON
json_path = os.path.join(temp_dir, 'exoplanets.json')
export_data = {
    'names': names[:5].tolist() if hasattr(names, 'tolist') else names[:5],
    'masses': masses[:5].to_dict(),
    'distances': distances[:5].to_dict(),
    'periods': periods[:5].to_dict()
}

with open(json_path, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"\nSaved to JSON: {json_path}")
print(f"File size: {os.path.getsize(json_path)} bytes")

In [ ]:
# Reload from JSON and verify units are preserved
with open(json_path, 'r') as f:
    loaded_data = json.load(f)

# Reconstruct DimArrays
loaded_masses = DimArray.from_dict(loaded_data['masses'])
loaded_distances = DimArray.from_dict(loaded_data['distances'])
loaded_periods = DimArray.from_dict(loaded_data['periods'])

print("\nLoaded from JSON:")
print(f"Masses: {loaded_masses}")
print(f"Distances: {loaded_distances}")
print(f"Periods: {loaded_periods}")

# Verify data integrity
assert np.allclose(loaded_masses._data, masses[:5]._data)
assert loaded_masses.unit == masses.unit
print("\n✓ JSON round-trip successful - data and units preserved!")

In [ ]:
# Save to pandas DataFrame
if HAS_PANDAS:
    # Create DataFrame with converted units for export
    export_df = pd.DataFrame({
        'name': names,
        'mass_mjup': masses._data,
        'mass_mearth': masses_mearth._data,
        'distance_au': distances._data,
        'period_days': periods._data,
        'period_years': periods_years._data,
        'radius_rjup': radii._data,
        'surface_gravity_ms2': gravity_ms2._data,
        'escape_velocity_kms': v_esc_kms._data
    })
    
    csv_path = os.path.join(temp_dir, 'exoplanets.csv')
    export_df.to_csv(csv_path, index=False)
    
    print(f"\nSaved to CSV: {csv_path}")
    print(f"File size: {os.path.getsize(csv_path)} bytes")
    print(f"\nFirst few rows:")
    print(export_df.head())
else:
    print("\npandas not available - skipping DataFrame export")

In [ ]:
# Save to HDF5 (if available)
try:
    import h5py
    
    hdf5_path = os.path.join(temp_dir, 'exoplanets.h5')
    
    with h5py.File(hdf5_path, 'w') as f:
        # Create datasets with attributes for units
        ds_mass = f.create_dataset('masses', data=masses._data)
        ds_mass.attrs['unit'] = str(masses.unit)
        
        ds_dist = f.create_dataset('distances', data=distances._data)
        ds_dist.attrs['unit'] = str(distances.unit)
        
        ds_period = f.create_dataset('periods', data=periods._data)
        ds_period.attrs['unit'] = str(periods.unit)
        
        # Save names as strings
        dt = h5py.string_dtype(encoding='utf-8')
        ds_names = f.create_dataset('names', data=names, dtype=dt)
    
    print(f"\nSaved to HDF5: {hdf5_path}")
    print(f"File size: {os.path.getsize(hdf5_path)} bytes")
    
except ImportError:
    print("\nh5py not available - skipping HDF5 export")

In [ ]:
# Summary of saved files
print("\n" + "="*60)
print("EXPORT SUMMARY")
print("="*60)
print(f"\nAll files saved to: {temp_dir}\n")

for filename in os.listdir(temp_dir):
    filepath = os.path.join(temp_dir, filename)
    size = os.path.getsize(filepath)
    print(f"  {filename:25s} {size:>8,} bytes")

print("\n" + "="*60)
print("\n✓ Data analysis complete!")
print(f"  - Loaded {len(masses)} exoplanets")
print(f"  - Performed unit conversions across 5 unit systems")
print(f"  - Calculated {len([masses, distances, periods, radii, surface_gravity, escape_velocity])} derived quantities")
if HAS_MATPLOTLIB:
    print(f"  - Generated 5 publication-quality plots")
print(f"  - Exported data in multiple formats")
print("  - All dimensional analysis checks passed")